In [1]:
#!pip install duckdb pandas requests

In [2]:
import requests
import zipfile
import io
import os
import pandas as pd


#Here we can try differnt years but i think the latest is 2024 ia to check and honestlly based on problem statement 
def download_pums_file(state_abbr, record_type, year="2023"):
    """
    record_type: 'p' for person, 'h' for housing
    """
    url = f"https://www2.census.gov/programs-surveys/acs/data/pums/{year}/5-Year/csv_{record_type}{state_abbr}.zip"
    dest_folder = f"data_{state_abbr}"
    os.makedirs(dest_folder, exist_ok=True)
    
    print(f"Downloading {record_type} records for {state_abbr}...")
    response = requests.get(url, stream=True)
    
    with zipfile.ZipFile(io.BytesIO(response.content)) as z:
        z.extractall(dest_folder)
        print(f"Extracted to {dest_folder}")

# Download  California we only want to do once really 
#download_pums_file("ca", "p")
#download_pums_file("ca", "h")

## Housing Specifically 

In [3]:
all_headers_housing = pd.read_csv('data_ca/psam_h06.csv', nrows=0).columns.tolist()
print(f"The length of Housing Features are: {len(all_headers_housing)}")
print("<------------------ These column names are the following ------------------>")
print(*all_headers_housing[:237])


The length of Housing Features are: 237
<------------------ These column names are the following ------------------>
RT SERIALNO DIVISION PUMA REGION STATE ADJHSG ADJINC WGTP NP TYPEHUGQ ACCESSINET ACR AGS BATH BDSP BLD BROADBND COMPOTHX CONP DIALUP ELEFP ELEP FS FULFP FULP GASFP GASP HFL HISPEED HOTWAT INSP LAPTOP MHP MRGI MRGP MRGT MRGX OTHSVCEX REFR RMSP RNTM RNTP RWAT RWATPR SATELLITE SINK SMARTPHONE SMP STOV TABLET TEL TEN VACS VALP VEH WATFP WATP YRBLT CPLT FINCP FPARC GRNTP GRPIP HHL HHLANP HHLDRAGEP HHLDRHISP HHLDRRAC1P HHT HHT2 HINCP HUGCL HUPAC HUPAOC HUPARC KIT LNGI MULTG MV NOC NPF NPP NR NRC OCPIP PARTNER PLM PLMPRP PSF R18 R60 R65 RESMODE SMOCP SMX SRNT SVAL TAXAMT WIF WKEXREL WORKSTAT FACCESSP FACRP FAGSP FBATHP FBDSP FBLDP FBROADBNDP FCOMPOTHXP FCONP FDIALUPP FELEP FFINCP FFSP FFULP FGASP FGRNTP FHFLP FHINCP FHISPEEDP FHOTWATP FINSP FKITP FLAPTOPP FMHP FMRGIP FMRGP FMRGTP FMRGXP FMVP FOTHSVCEXP FPLMP FPLMPRP FREFRP FRMSP FRNTMP FRNTP FRWATP FRWATPRP FSATELLITEP FSINKP F

In [4]:
# Note that htere are some that we dont een need here... I am also not cleaning this in the best way possible 
clean_housing_cols = [c for c in all_headers_housing if not c.startswith('WGTP') and not c.startswith('F')]
print("<------------------ The 'Clean' column names are the following ------------------>")
print(*clean_housing_cols[:237])

<------------------ The 'Clean' column names are the following ------------------>
RT SERIALNO DIVISION PUMA REGION STATE ADJHSG ADJINC NP TYPEHUGQ ACCESSINET ACR AGS BATH BDSP BLD BROADBND COMPOTHX CONP DIALUP ELEFP ELEP GASFP GASP HFL HISPEED HOTWAT INSP LAPTOP MHP MRGI MRGP MRGT MRGX OTHSVCEX REFR RMSP RNTM RNTP RWAT RWATPR SATELLITE SINK SMARTPHONE SMP STOV TABLET TEL TEN VACS VALP VEH WATFP WATP YRBLT CPLT GRNTP GRPIP HHL HHLANP HHLDRAGEP HHLDRHISP HHLDRRAC1P HHT HHT2 HINCP HUGCL HUPAC HUPAOC HUPARC KIT LNGI MULTG MV NOC NPF NPP NR NRC OCPIP PARTNER PLM PLMPRP PSF R18 R60 R65 RESMODE SMOCP SMX SRNT SVAL TAXAMT WIF WKEXREL WORKSTAT


##### we can try playing with this later 
import duckdb
###Connect and view the first 5 rows of specific columns
con = duckdb.connect()
sample_query = f"SELECT {', '.join(clean_housing_cols[:10])} FROM 'data_ca/psam_h06.csv' LIMIT 10"
print(con.execute(sample_query).df())

In [5]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df_clean_housing = pd.read_csv('data_ca/psam_h06.csv',usecols=clean_housing_cols)
df_clean_housing.head(5)

,RT,SERIALNO,DIVISION,PUMA,REGION,STATE,ADJHSG,ADJINC,NP,TYPEHUGQ,ACCESSINET,ACR,AGS,BATH,BDSP,BLD,BROADBND,COMPOTHX,CONP,DIALUP,ELEFP,ELEP,GASFP,GASP,HFL,HISPEED,HOTWAT,INSP,LAPTOP,MHP,MRGI,MRGP,MRGT,MRGX,OTHSVCEX,REFR,RMSP,RNTM,RNTP,RWAT,RWATPR,SATELLITE,SINK,SMARTPHONE,SMP,STOV,TABLET,TEL,TEN,VACS,VALP,VEH,WATFP,WATP,YRBLT,CPLT,GRNTP,GRPIP,HHL,HHLANP,HHLDRAGEP,HHLDRHISP,HHLDRRAC1P,HHT,HHT2,HINCP,HUGCL,HUPAC,HUPAOC,HUPARC,KIT,LNGI,MULTG,MV,NOC,NPF,NPP,NR,NRC,OCPIP,PARTNER,PLM,PLMPRP,PSF,R18,R60,R65,RESMODE,SMOCP,SMX,SRNT,SVAL,TAXAMT,WIF,WKEXREL,WORKSTAT
0,H,2019GQ0000003,9,3704,4,6,1195583,1207712,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,H,2019GQ0000009,9,7322,4,6,1195583,1207712,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,H,2019GQ0000013,9,5923,4,6,1195583,1207712,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,H,2019GQ0000023,9,7107,4,6,1195583,1207712,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,H,2019GQ0000024,9,8900,4,6,1195583,1207712,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:

missing_values_count = df_clean_housing.isna().sum()
print(missing_values_count)

RT                 0
SERIALNO           0
DIVISION           0
PUMA               0
REGION             0
STATE              0
ADJHSG             0
ADJINC             0
NP                 0
TYPEHUGQ           0
ACCESSINET    131474
ACR           280428
AGS           741057
BATH           91660
BDSP           91660
BLD            91660
BROADBND      176402
COMPOTHX      131474
CONP          380550
DIALUP        176402
ELEFP         131474
ELEP          160685
GASFP         131474
GASP          341320
HFL           131474
HISPEED       176402
HOTWAT         91660
INSP          380550
LAPTOP        131474
MHP           768886
MRGI          526554
MRGP          526554
MRGT          526554
MRGX          380550
OTHSVCEX      176402
REFR           91660
RMSP           91660
RNTM          538403
RNTP          538403
RWAT           91660
RWATPR         91660
SATELLITE     176402
SINK           91660
SMARTPHONE    131474
SMP           752532
STOV           91660
TABLET        131474
TEL          

## Persons Specifically 

In [7]:

all_headers_persons = pd.read_csv('data_ca/psam_p06.csv', nrows=0).columns.tolist()
print("<------------------ These column names are the following ------------------>")
print(f"The length of Person Variables are: {len(all_headers_persons)}")
print(*all_headers_persons[:286])


<------------------ These column names are the following ------------------>
The length of Person Variables are: 286
RT SERIALNO DIVISION SPORDER PUMA REGION STATE ADJINC PWGTP AGEP CIT CITWP COW DDRS DEAR DEYE DOUT DPHY DRAT DRATX DREM ENG FER GCL GCM GCR HINS1 HINS2 HINS3 HINS4 HINS5 HINS6 HINS7 INTP JWMNP JWRIP JWTRNS LANX MAR MARHD MARHM MARHT MARHW MARHYP MIG MIL MLPA MLPB MLPCD MLPE MLPFG MLPH MLPIK MLPJ NWAB NWAV NWLA NWLK NWRE OIP PAP RELSHIPP RETP SCH SCHG SCHL SEMP SEX SSIP SSP WAGP WKHP WKL WKWN WRK YOEP ANC ANC1P ANC2P DECADE DIS DRIVESP ESP ESR FOD1P FOD2P HICOV HISP INDP JWAP JWDP LANP MIGPUMA MIGSP MSP NAICSP NATIVITY NOP OC OCCP PAOC PERNP PINCP POBP POVPIP POWPUMA POWSP PRIVCOV PUBCOV QTRBIR RAC1P RAC2P19 RAC2P23 RAC3P RACAIAN RACASN RACBLK RACNH RACNUM RACPI RACSOR RACWHT RC SCIENGP SCIENGRLP SFN SFR SOCP VPS WAOB FAGEP FANCP FCITP FCITWP FCOWP FDDRSP FDEARP FDEYEP FDISP FDOUTP FDPHYP FDRATP FDRATXP FDREMP FENGP FESRP FFERP FFODP FGCLP FGCMP FGCRP FHICOVP FHINS1P FHIN

In [8]:
#quick and dirty column clean, this is just  a list though 
clean_persons_cols = [c for c in all_headers_persons if not c.startswith('PWG') and not c.startswith('F')]
print("<------------------ The 'Clean' column names are the following ------------------>")
print(*clean_persons_cols[:286])

<------------------ The 'Clean' column names are the following ------------------>
RT SERIALNO DIVISION SPORDER PUMA REGION STATE ADJINC AGEP CIT CITWP COW DDRS DEAR DEYE DOUT DPHY DRAT DRATX DREM ENG GCL GCM GCR HINS1 HINS2 HINS3 HINS4 HINS5 HINS6 HINS7 INTP JWMNP JWRIP JWTRNS LANX MAR MARHD MARHM MARHT MARHW MARHYP MIG MIL MLPA MLPB MLPCD MLPE MLPFG MLPH MLPIK MLPJ NWAB NWAV NWLA NWLK NWRE OIP PAP RELSHIPP RETP SCH SCHG SCHL SEMP SEX SSIP SSP WAGP WKHP WKL WKWN WRK YOEP ANC ANC1P ANC2P DECADE DIS DRIVESP ESP ESR HICOV HISP INDP JWAP JWDP LANP MIGPUMA MIGSP MSP NAICSP NATIVITY NOP OC OCCP PAOC PERNP PINCP POBP POVPIP POWPUMA POWSP PRIVCOV PUBCOV QTRBIR RAC1P RAC2P19 RAC2P23 RAC3P RACAIAN RACASN RACBLK RACNH RACNUM RACPI RACSOR RACWHT RC SCIENGP SCIENGRLP SFN SFR SOCP VPS WAOB


In [9]:
# This is breaking cant even convert to a df this is whre duck db comes into play or 
#df_clean_persons_ = pd.read_csv('data_ca/psam_p06.csv')

#print(df_clean_persons_.head())

In [10]:
# This may or may not work MIGHT BREAK KERNEL (hence why we may need to not use pandas and maybe use something like spark, or duck db
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
df_clean_persons = pd.read_csv('data_ca/psam_p06.csv',usecols=clean_persons_cols)
df_clean_persons.head(1)

,RT,SERIALNO,DIVISION,SPORDER,PUMA,REGION,STATE,ADJINC,AGEP,CIT,CITWP,COW,DDRS,DEAR,DEYE,DOUT,DPHY,DRAT,DRATX,DREM,ENG,GCL,GCM,GCR,HINS1,HINS2,HINS3,HINS4,HINS5,HINS6,HINS7,INTP,JWMNP,JWRIP,JWTRNS,LANX,MAR,MARHD,MARHM,MARHT,MARHW,MARHYP,MIG,MIL,MLPA,MLPB,MLPCD,MLPE,MLPFG,MLPH,MLPIK,MLPJ,NWAB,NWAV,NWLA,NWLK,NWRE,OIP,PAP,RELSHIPP,RETP,SCH,SCHG,SCHL,SEMP,SEX,SSIP,SSP,WAGP,WKHP,WKL,WKWN,WRK,YOEP,ANC,ANC1P,ANC2P,DECADE,DIS,DRIVESP,ESP,ESR,HICOV,HISP,INDP,JWAP,JWDP,LANP,MIGPUMA,MIGSP,MSP,NAICSP,NATIVITY,NOP,OC,OCCP,PAOC,PERNP,PINCP,POBP,POVPIP,POWPUMA,POWSP,PRIVCOV,PUBCOV,QTRBIR,RAC1P,RAC2P19,RAC2P23,RAC3P,RACAIAN,RACASN,RACBLK,RACNH,RACNUM,RACPI,RACSOR,RACWHT,RC,SCIENGP,SCIENGRLP,SFN,SFR,SOCP,VPS,WAOB
0,P,2019GQ0000003,9,1,3704,4,6,1207712,58,1,NaN,1.0,1.0,1,2,1.0,1.0,NaN,NaN,1.0,NaN,2.0,NaN,NaN,2,2,1,2,2,2,2,0.0,NaN,NaN,NaN,2.0,5,NaN,NaN,NaN,NaN,NaN,1.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,1.0,3.0,0.0,0.0,38,0.0,1.0,NaN,18.0,0.0,2,0.0,0.0,23100.0,20.0,1.0,32.0,2.0,NaN,3,995,999,NaN,1,NaN,NaN,3.0,1,1,6190.0,NaN,NaN,NaN,NaN,NaN,6.0,4853,1,NaN,NaN,9142.0,NaN,23100.0,23100.0,17,175.0,NaN,NaN,2,1,2,2,2,-999,2,0,0,1,0,1,0,0,0,NaN,NaN,NaN,NaN,NaN,533054,NaN,1


In [11]:
# Size of data 
!du -h data_ca/ | sort -h

1.5G	data_ca/


###### I guess we can think about the actual architecture or ways of manipulating the file, and agregating everything into a single df (if we even wanna do that) 

import duckdb

###### connection to a local DuckDB file (this acts as  storage)
con = duckdb.connect('acs_california_2023.db')

######   Register the CSVs as virtual tablesm ??? (without loading them into RAM)

con.execute("""
    CREATE TABLE person_raw AS SELECT * FROM read_csv_auto('data_ca/psam_p06.csv');
    CREATE TABLE housing_raw AS SELECT * FROM read_csv_auto('data_ca/psam_h06.csv');
""")

######  Join  and select only the columns needed, here we must figure out what we acually want to use ... to keep table small
con.execute("""
    CREATE TABLE ml_ready_data AS 
    SELECT 
        p.SERIALNO, 
        p.AGEP, 
        p.SEX, 
        p.SCHL, 
        p.ESR, -- Employment status
        h.HINCP, -- Household Income
        h.NP,    -- Number of persons in household
        h.VEH    -- Number of vehicles
    FROM person_raw p
    JOIN housing_raw h ON p.SERIALNO = h.SERIALNO
""")

print(con.execute("SELECT count(*) FROM ml_ready_data").fetchall())